# Fiber Network

The problem provided in this example is a fiber network with fixed-fixed (both displacement and moments) boundary conditions with a prescribed Tensile displacement. (i.e. nonhomogenous Dirichlet Boundary Condition) on the top boundary. Each fiber is modeled with 2D geometrically exact beams (i.e. Simo-Reissner Beams).

In [ ]:
from dolfin import *
import matplotlib.pyplot as plt
import numpy as np
import meshio

# Load the mesh from the XDMF file
mesh = Mesh()
with XDMFFile("voronoi_nxt_v7.xdmf") as infile:
    infile.read(mesh)

# Load boundary tags
mvc = MeshValueCollection("size_t", mesh, 0)  # Point data (vertices)
with XDMFFile("voronoi_nxt_v7.xdmf") as infile:
    infile.read(mvc, "boundary_tags")
boundary_tags = MeshFunction("size_t", mesh, mvc)

# Get the mesh vertices and their coordinates
mesh_vertices = mesh.coordinates()
num_mesh_vertices = len(mesh_vertices)

# Load the raw boundary tags directly from the XDMF file using meshio
meshio_mesh = meshio.read("voronoi_nxt_v7.xdmf")
raw_boundary_tags = meshio_mesh.point_data["boundary_tags"]

# Reindex boundary tags to match the mesh vertices
tags = np.zeros(num_mesh_vertices, dtype=int)
if len(raw_boundary_tags) == num_mesh_vertices:
    tags[:] = raw_boundary_tags
else:
    raise ValueError(f"Mismatch: raw_boundary_tags ({len(raw_boundary_tags)}) does not match mesh vertices ({num_mesh_vertices})")

# Diagnostic: Check array sizes
vertices = mesh.coordinates()
print(f"Number of vertices: {len(vertices)}")
print(f"Length of boundary_tags (raw): {len(raw_boundary_tags)}")
print(f"Length of tags (after reindexing): {len(tags)}")

# Plot tagged nodes
plt.figure(figsize=(8, 8), facecolor='white')
plot(mesh, color='blue', linewidth=0.5, alpha=0.6, title="Voronoi Mesh")
bottom_nodes = vertices[tags == 1]
top_nodes = vertices[tags == 2]
plt.scatter(bottom_nodes[:, 0], bottom_nodes[:, 1], c='red', label='Bottom (tag 1)', s=50)
plt.scatter(top_nodes[:, 0], top_nodes[:, 1], c='green', label='Top (tag 2)', s=50)

# Customize the plot
plt.xlabel("x", fontsize=12)
plt.ylabel("y", fontsize=12)
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.grid(True, linestyle='--', alpha=0.3)
plt.gca().set_facecolor('#f5f5f5')
plt.legend()

# Display the plot
plt.show()

%matplotlib inline
from dolfin import *
import numpy as np
import matplotlib.pyplot as plt
from displacement_control_solver import displacement_control

try:
    from ufl import diag, Jacobian, shape
except:
    from ufl_legacy import diag, Jacobian, shape

parameters["form_compiler"]["cpp_optimize"] = True
parameters["form_compiler"]["quadrature_degree"] = 3
parameters['reorder_dofs_serial'] = False

ffc_options = {"optimize": True, \
               "eliminate_zeros": True, \
               "precompute_basis_const": True, \
               "precompute_ip_const": True}

## Import Mesh and define function spaces

In the case of 2D beams we also define the rotation matrix about the $z$ axis and directional derivative with respect to the beam centerline.

In [ ]:
mesh = Mesh()
with XDMFFile('voronoi_nxt_v7.xdmf') as infile:  # Updated filename
    infile.read(mesh)

# Diagnostic: Check mesh coordinate range
print(f"Mesh coordinate range: x: [{mesh.coordinates()[:, 0].min():.2f}, {mesh.coordinates()[:, 0].max():.2f}], "
      f"y: [{mesh.coordinates()[:, 1].min():.2f}, {mesh.coordinates()[:, 1].max():.2f}]")

Ue = VectorElement("CG", mesh.ufl_cell(), 2, dim=2)  # displacement
Te = FiniteElement("CG", mesh.ufl_cell(), 1)  # rotation

V = FunctionSpace(mesh, MixedElement([Ue, Te]))   

v_ = TestFunction(V)
u_, theta_ = split(v_)
dv = TrialFunction(V)
v = Function(V, name="Generalized displacement")
u, theta = split(v)

VR = TensorFunctionSpace(mesh, "DG", 0, shape=(2, 2))
V0 = FunctionSpace(mesh, "DG", 0)

Vu = V.sub(0).collapse()
disp = Function(Vu)

# Jacobian and geometric quantities
Jac = Jacobian(mesh)
Jac_2d = as_vector([Jac[0, 0], Jac[1, 0]])  # Use only x, y components
g01 = Jac_2d / sqrt(dot(Jac_2d, Jac_2d))
g02 = as_vector([-g01[1], g01[0]])

r01 = outer(g01, as_vector([1, 0]))
r02 = outer(g02, as_vector([0, 1]))
R0 = r01 + r02

# Define functions for beams
def tgrad(u):  # directional derivative w.r.t. beam centerline
    return dot(grad(u), g01)

def rotation_matrix(theta):  # 2D rotation matrix
    return as_tensor([[cos(theta), -sin(theta)], [sin(theta), cos(theta)]])
Rot = rotation_matrix(theta)

## Define Dirichlet Boundary Conditions

**Note that for the case of displacement control, the FEniCS expression for the applied displacement must be positive to prevent convergence issues.**

For example:

```
apply_disp = Expression("t", t = 0.0, degree = 0)
```

is valid and will not have convergence issues while

```
apply_disp = Expression("-t", t = 0.0, degree = 0)
```

can cause convergence issues.

The direction of applied loading will be determined by the initial load step.

In [ ]:
H = 100.0
w = 100.0

# Use the raw boundary tags loaded in Cell 1
# Create a MeshFunction for boundary conditions
boundary_tags_func = MeshFunction("size_t", mesh, 0)
boundary_tags_func.set_values(tags)  # Use the reindexed tags from Cell 1

# Diagnostic: Confirm the number of tagged vertices
num_bottom = np.sum(tags == 1)
num_top = np.sum(tags == 2)
print(f"Number of bottom vertices (tag 1): {num_bottom}")
print(f"Number of top vertices (tag 2): {num_top}")

# Define boundary conditions using the MeshFunction with the topological method
BC_bot = DirichletBC(V, Constant((0.0, 0.0, 0.0)), boundary_tags_func, 1)  # Fixed displacement and rotation
BC_top_x = DirichletBC(V.sub(0).sub(0), Constant(0.0), boundary_tags_func, 2)  # Fix x-displacement
BC_top_rot = DirichletBC(V.sub(1), Constant(0.0), boundary_tags_func, 2)  # Fix rotation

apply_disp = Expression("t", t=0.0, degree=0)  # Expression to compress the top
BC_top_y = DirichletBC(V.sub(0).sub(1), apply_disp, boundary_tags_func, 2)  # Incrementally compress the top in y

bcs = [BC_bot, BC_top_y, BC_top_rot, BC_top_x]

# Diagnostic: Check boundary application
for bc in bcs:
    boundary_values = bc.get_boundary_values()
    print(f"Boundary condition {bc}: {len(boundary_values)} DoFs constrained")

## Kinematics and Weak Form

In [ ]:
# Define tgrad for 1D elements
def tgrad(u):
    if u.ufl_shape == (2,):  # For vector u (displacement)
        grad_u = grad(u)  # Shape: (2, 2)
        return as_vector([sum(grad_u[i, j] * g01[j] for j in range(2)) for i in range(2)])
    else:  # For scalar theta (rotation)
        grad_theta = grad(u)  # Shape: (2,)
        return sum(grad_theta[j] * g01[j] for j in range(2))

# Kinematics: Total beam formulation
defo = dot(R0.T, dot(Rot.T, g01 + tgrad(u)) - g01)
curv = tgrad(theta)

In [ ]:
# Geometrical properties
S = 1.5*3 # cross-sectional area
I = 3*1.5**3/12 # Area moment
G = 0.0412 # Shear Modulus
nu = 0.5
E = 2*G*(1+nu)

kappa = 5*(1+nu)/(6+5*nu) # Shear correction (Timoshenko)

# Stiffness moduli
ES = E*S
GS = G*kappa*S
EI = E*I

In [ ]:
# Constitutive Equations
C_N = diag(as_vector([ES, GS]))

# Applied Load:
F_max = Constant((0.0,0.0))
M_max = Constant(0.0)

elastic_energy = 0.5 * (dot(defo, dot(C_N, defo)) + (EI*curv**2))*dx

F_int = derivative(elastic_energy, v, v_)
F_ext =(-M_max*theta_ + dot(F_max, u_)) * ds
residual = F_int - F_ext
tangent_form = derivative(residual, v, dv)

### Using the solver
1. Initialize the solver by calling solver.initialize()
2. Iteratively call solver.solve() until desired stopping condition

In [ ]:
# Solver Parameters
psi = 1.0
abs_tol = 1.0e-6
lmbda0 = 0.5 # Positive for Stretch
max_iter = 20
solver = 'mumps'

# Set up arc-length solver
solver = displacement_control(psi=psi, lmbda0=lmbda0, max_iter=max_iter, u=v,
                       F_int=F_int, F_ext=F_ext, bcs=bcs, J=tangent_form, displacement_factor = apply_disp, solver = solver)

In [ ]:
disp = [v.vector()[:]]
lmbda = [0]
# Function space to compute reaction force at each iteration
v_reac = Function(V)
bcRy = DirichletBC(V.sub(0).sub(1), Constant(1.0), boundary_tags_func, 1, method="topological")  # Reaction force at bottom
f_reac = [0.0]
for ii in range(0, 20):
    print(f"Step {ii}, apply_disp.t = {apply_disp.t}")
    solver.solve()
    if solver.converged:
        # Store whole displacement field
        disp.append(v.vector()[:])
        # Store displacement factor
        lmbda.append(apply_disp.t)
        # Compute and store reaction force
        bcRy.apply(v_reac.vector())
        f_reac.append(assemble(action(residual, v_reac)))

In [ ]:
plt.figure(figsize=(7,5))
plt.plot(np.array(lmbda), -np.array(f_reac), c='k', marker='o')  # Negate f_reac to make it positive
plt.xlabel('Applied Displacement (Stretching, +y)')
plt.ylabel('Reaction Force at Bottom (Magnitude)')
plt.title('Equilibrium Path (Stretching)')
plt.grid(True)
plt.show()

## Optional: Creating an animation from solution snapshots

In [ ]:
from matplotlib import animation, rc
import matplotlib.pyplot as plt
import numpy as np

# Define nodal coordinates using dof_coords
x_nodal_coord = dof_coords[x_dofs, 0]  # Shape: (2748,)
y_nodal_coord = dof_coords[x_dofs, 1]  # Shape: (2748,)

# Set up animation
plt.rcParams["animation.html"] = "jshtml"

fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111)

# Adjust y-axis limits for stretching
w = 100.0  # From Cell 3
H = 100.0  # From Cell 3
ax.set_xlim([0, w])
ax.set_ylim([0, H+20])  # Adjusted for stretching (upward displacement)

# Plot initial and deformed configurations
deformed, = ax.plot([], [], lw=7, c='r', label='Deformed Configuration', ls='None', marker='.')
init, = ax.plot(x_nodal_coord, y_nodal_coord, c='k', lw=5, ls='None', label='Initial Configuration', marker='.', alpha=0.3)
ax.legend(loc='lower right')

def drawframe(n):
    disp_x = x_nodal_coord + disp[n][x_dofs]
    disp_y = y_nodal_coord + disp[n][y_dofs]
    
    deformed.set_data(disp_x, disp_y)
    return deformed,

plt.close()
anim = animation.FuncAnimation(fig, drawframe, frames=len(lmbda), interval=40, blit=True)

anim